# **Nandan D | PES2UG23CS363 | F**
## LangChain Assignment

---



# Unit 2 - Part 3a: Chain of Thought (CoT)

In [2]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.0 MB/s eta 0:00:00


In [3]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


# Concept: Latent Reasoning

In [4]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


# The Experiment: A Tricky Math Problem

In [5]:
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


# Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

In [6]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

# COT

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a child psychologist, I would recommend Solution 3: Create a "Veggie Face" on Their Plate as the most sustainable approach to encouraging a 5-year-old to eat vegetables. Here's why:

1. **Involves the child in the process**: This approach encourages the child to participate in creating a fun and engaging meal, which fosters a sense of ownership and control over the experience. By letting the child decide how the face looks, they feel more invested in the meal and are more likely to try the vegetables.
2. **Develops problem-solving skills**: Creating a veggie face requires the child to think creatively and come up with solutions to fit the vegetables together. This helps develop their problem-solving skills and encourages critical thinking.
3. **Builds self-esteem**: By letting the child take the lead in creating the veggie face, they feel proud of their creation and more confident in their ability to try new foods.
4. **Promotes a positive relat

# GOT


In [8]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
Here's a new movie idea that combines the technology of Sci-Fi, the passion of Romance, and the fear of Horror:

"Moonfall" follows Dr. Emma Taylor, a brilliant and reclusive physicist who has spent her entire career studying the mysteries of time. When she finally cracks the code to time travel, she's ecstatic, but her excitement is short-lived as she discovers a way to manipulate the fabric of time itself. As she experiments with her invention, she meets Ryan, a charming and handsome pilot who has been sent back in time from a dystopian future where the world is on the brink of collapse. As they navigate through the complexities of time travel, they realize that their love is the key to preventing the apocalypse, but a malevolent entity, born from the darkness of the future, begins to hunt them down, threatening to destroy not just their love, but the very fabric of reality itself. As they fight to survive and prevent the impending doom, they mu